In [1]:
# Input handling

with open("Day7_input.txt", 'r') as input_file:
    input_raw = input_file.read().split('\n')

start_raw = input_raw[0]
maze_raw = input_raw[1:]

# Coordinates of the starting beam
start_beam = set([i for i, beam in enumerate(start_raw) if beam == 'S'])

# Maze processing
maze = []

for row_i, row in enumerate(maze_raw):
    maze_row = []
    for col_i, point in enumerate(row):
        if point == '^':
            maze_row.append(col_i)
    maze.append(maze_row)

In [2]:
maze_shape = {'x': len(maze_raw[0]),
              'y': len(maze_raw)}

In [3]:
def move_beam_single_row(beams, maze, row_number=0, collisions=0):

    new_beams = beams.copy()

    for beam in beams:
        if beam in maze[row_number]:
            # print(f'Beam at x {beam} encountered a splitter!')
            collisions += 1
            new_beams.remove(beam)
            new_beams.add(beam-1)
            new_beams.add(beam+1)
    
    return new_beams, collisions
    

In [4]:
def move_beam_through_maze(beam, maze, row_number=0, collisions=0):

    new_beam, collisions = move_beam_single_row(beam, maze, row_number, collisions)
    
    if row_number == len(maze)-1:
        return new_beam, collisions
    
    else:
        return move_beam_through_maze(new_beam, maze, row_number+1, collisions)

In [5]:
end_beam, total_collisions = move_beam_through_maze(start_beam, maze)
print(f'Beams at the end: {end_beam}')
print(f'We had {total_collisions} collisions')

Beams at the end: {0, 2, 3, 4, 5, 6, 8, 10, 11, 12, 14, 16, 17, 18, 20, 22, 23, 25, 27, 29, 30, 32, 33, 34, 36, 38, 40, 42, 43, 44, 46, 47, 48, 50, 52, 54, 56, 57, 58, 60, 62, 63, 65, 66, 68, 69, 70, 72, 74, 76, 78, 79, 80, 82, 83, 85, 87, 88, 90, 91, 92, 94, 96, 98, 99, 100, 101, 103, 104, 106, 107, 109, 110, 112, 114, 116, 117, 119, 120, 122, 123, 125, 127, 129, 130, 132, 133, 134, 136, 138, 140}
We had 1555 collisions


Part 2

In [6]:
# # Input handling

# with open("Day7_ex_input.txt", 'r') as input_file:
#     input_raw = input_file.read().split('\n')

# start_raw = input_raw[0]
# maze_raw = input_raw[1:]

# # Coordinates of the starting beam
# start_beam = start_raw.index('S')

# # Maze processing
# maze = []

# for row_i, row in enumerate(maze_raw):
#     maze_row = []
#     for col_i, point in enumerate(row):
#         if point == '^':
#             maze_row.append(col_i)
#     maze.append(maze_row)

In [7]:
# def trace_beam_through_maze(beam, maze, row_number=0):

#     # Base case
#     if row_number == len(maze):
#         return  1
    
#     # Recursion
#     else:
#         if beam in maze[row_number]:
#             # print(f'Beam at x {beam} encountered a splitter!')
#             beam_left = beam-1
#             beam_right = beam+1

#             return trace_beam_through_maze(beam_left, maze, row_number+1) + trace_beam_through_maze(beam_right, maze, row_number+1)

#         else:

#             return trace_beam_through_maze(beam, maze, row_number+1)

Making a tree

In [8]:
# class Node:
#     def __init__(self, coordinate):
#         self.left = None
#         self.right = None
#         self.coordinate = coordinate

#     def add_left(self, left_node):
#         self.left = left_node
    
#     def add_right(self, right_node):
#         self.right = right_node

#     def print_tree(self, level=0):

#         print(f'Level {level}: {self.coordinate}')

#         if self.left:
#             self.left.print_tree(level = level+1)
#         if self.right:
#             self.right.print_tree(level = level+1)

In [9]:
# def populate_maze_tree(beam, maze, row_number=0, node_count=0):

#     # Base case
#     if row_number == len(maze):
#         return node_count
    
#     # Recursion

#     x = beam.coordinate
#     node_list={x: beam}

#     if x in maze[row_number]:
#         # print(f'Beam at x {beam} encountered a splitter!')
#         if x-1 in node_list:
#             beam.add_left(node_list[x-1])
#         else:
#             left = Node(x-1)
#             node_list[x-1] = left
#             beam.add_left(left)
#             populate_maze_tree(left, maze, row_number+1)

#         if x+1 in node_list:
#             beam.add_right(node_list[x+1])
#         else:
#             right = Node(x+1)
#             node_list[x+1] = right
#             beam.add_right(right)
#             populate_maze_tree(right, maze, row_number+1)
        
#     else:
#         populate_maze_tree(beam, maze, row_number+1, node_count)

In [10]:
# beam = Node(start_beam)
# print(populate_maze_tree(beam, maze))

Try with dicts

In [1]:
# Input handling

with open("Day7_input.txt", 'r') as input_file:
    input_raw = input_file.read().split('\n')

start_raw = input_raw[0]
maze_raw = input_raw[1:]

# Starting beam row as a dict, with possible paths there as a value
start_beam={}
for x, char in enumerate(start_raw):
    if char == 'S':
        start_beam[x] = 1
    else:
        start_beam[x] = 0


# Maze processing
maze = []

for row_i, row in enumerate(maze_raw):
    maze_row = []
    for col_i, point in enumerate(row):
        if point == '^':
            maze_row.append(col_i)
    maze.append(maze_row)

In [32]:
def trace_beam_single_row(beams, maze, row_number=0):

    new_beams = beams.copy()

    for beam in filter(lambda item: item[1]>0, beams.items()):
        x = beam[0]
        paths = beam[1]
        if x in maze[row_number]:
            # print(f'Beam at x {beam} encountered a splitter!')
            new_beams[x-1] = new_beams[x-1] + paths
            new_beams[x+1] = new_beams[x+1] + paths
            new_beams[x] = 0
    print(' '.join(maze_raw[row_number]))
    str_new_beams = [str(value) for value in list(new_beams.values())]
    print(" ".join(str_new_beams))
    
    return new_beams

In [20]:
def trace_beam_through_maze(beam, maze, row_number=0):

    new_beam = trace_beam_single_row(beam, maze, row_number)
    
    if row_number == len(maze)-1:
        return new_beam
    
    else:
        return trace_beam_through_maze(new_beam, maze, row_number+1)

In [34]:
#print(list(start_beam.keys()))
print(' '.join(start_raw))
new_beams = trace_beam_through_maze(start_beam, maze)

print(sum(list(new_beams.values())))

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . S . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ^ . . . . . . 